In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas import Series, DataFrame

In [33]:
sdAttr = pd.read_csv("SanDisk_Attrition_Original.csv")

In [34]:
sdAttr.shape

(1340, 51)

**Data Cleansing**

In [35]:
# The following data cleaning steps can be run:

# 1) Remove duplicate rows from the given dataset
# 2) Round up the Risk Factor column to 2 decimal places\n
# 3) Drop duplicate rows again

In [36]:
# Step 1: Drops the duplicate rows
sdAttr_1 = sdAttr.drop_duplicates()

In [37]:
sdAttr_1.shape

(1232, 51)

In [38]:
# Step 2a: Rounding up the Risk Factors to two decimal places
RefRf = sdAttr_1['Rf'].apply(lambda x: round(x, 2))

In [39]:
# Step 2b: Adding a new column for the new Risk Factor
sdAttr_1['NewRf'] = RefRf

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
sdAttr_1.shape

(1232, 52)

In [41]:
# Step 2c: Drop the 'Rf' column
sdAttr_1.drop(['Rf'], axis = 1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [42]:
# Step 2d: Rename the column 'NewRf' to 'Rf'
sdAttr_1.rename(columns = {'NewRf': 'Rf'}, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
# Step 3: Obtain the final dataset after dropping duplicates again
sdAttrFinal = sdAttr_1.drop_duplicates()

In [44]:
sdAttrFinal.shape

(1196, 51)

**Data Analysis**

In [45]:
# Attrition is defined as the process of employees leaving the organization

# Attrition rate or percent is defined as the percentage of employees
# the organization lost and did not replace

In [46]:
# Some analysis that could not be covered using Tableau are covered here.

# One of our primary objective is to find out the Attrition rate year-on-year.

In [47]:
# Number of unique employees in each year
YrIdGp = sdAttrFinal.groupby("Year")["Id"].unique()

In [48]:
# A series of employees year-wise
print(YrIdGp)

Year
2012    [181994, 185184, 189491, 11410, 186257, 181955...
2013    [11410, 185891, 186257, 185949, 186056, 181445...
2014    [185891, 189498, 11192, 185949, 186056, 181445...
Name: Id, dtype: object


In [49]:
YearTotEmp = []
YearTotEmp.extend([len(YrIdGp[2012]), len(YrIdGp[2013]), len(YrIdGp[2014])])

In [50]:
print(YearTotEmp)

[405, 374, 344]


In [51]:
# Method 'setdiff1d' gives the elements in list1 that are not present in list2
AttrCountYear = [0]

AttrCountYear.append(len(np.setdiff1d(YrIdGp[2012], YrIdGp[2013])))
AttrCountYear.append(len(np.setdiff1d(YrIdGp[2013], YrIdGp[2014])))

In [52]:
# The list contains the attrition count year-on-year
AttrCountYear

[0, 206, 192]

In [53]:
# The number of original employees at the starting of the year
YrOrigEmp = []
YrOrigEmp.extend([YearTotEmp[0],
                  YearTotEmp[0] - AttrCountYear[1],
                  YearTotEmp[1] - AttrCountYear[2]])

In [54]:
# List containing the number of employees at the starting of
# each year after deducting the attritions of the previous year.
YrOrigEmp

[405, 199, 182]

In [55]:
# The number of hires in each of the years
YrHires = []
index = 0

while(index < 3):
  YrHires.append(YearTotEmp[index] - YrOrigEmp[index])
  index = index + 1

In [56]:
# List containing the number of hires in each year
YrHires

[0, 175, 162]

In [57]:
# Calculating the attrition rate

# Formula for calculating the attrition rate = 
# ((Number of employees who left in the period) / (Average of the number of employees in the period)) * 100

AttrRateList = [0]
index = 1

while (index < 3):
  AttrRate = ((YearTotEmp[index - 1] - YearTotEmp[index]) /
              ((YearTotEmp[index - 1] + YearTotEmp[index]) / 2)) * 100
  AttrRateList.append(round(AttrRate, 2))
  index = index + 1


In [58]:
# Attrition Rate year-on-year
AttrRateList

[0, 7.96, 8.36]

In [59]:
# Creating a table from the data
AttrDf = DataFrame({'Orig': YrOrigEmp,
                    'Hires': YrHires,
                    'Total': YearTotEmp,
                    'Attr Rate': AttrRateList})

In [31]:
# Attrition Table
AttrDf

,Orig,Hires,Total,Attr Rate
0,405,0,405,0.00
1,199,175,374,7.96
2,182,162,344,8.36
